# Rakentaminen Meta-perheen mallien kanssa

## Johdanto

Tässä osiossa käsitellään:

- Tutustutaan kahteen päämalliin Meta-perheestä – Llama 3.1 ja Llama 3.2
- Ymmärretään kunkin mallin käyttötarkoitukset ja tilanteet
- Koodiesimerkki, joka havainnollistaa kummankin mallin erityispiirteitä

## Meta-malliperhe

Tässä osiossa tutustumme kahteen Meta-perheen eli "Llama-lauman" malliin – Llama 3.1 ja Llama 3.2

Näistä malleista on saatavilla eri versioita, ja ne löytyvät Github Model -markkinapaikalta. Lisätietoa Github-mallien käytöstä [AI-mallien prototyyppien rakentamiseen](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Malliversiot:
- Llama 3.1 – 70B Instruct
- Llama 3.1 – 405B Instruct
- Llama 3.2 – 11B Vision Instruct
- Llama 3.2 – 90B Vision Instruct

*Huom: Llama 3 on myös saatavilla Github-malleissa, mutta sitä ei käsitellä tässä osiossa*


## Llama 3.1

405 miljardin parametrin Llama 3.1 kuuluu avoimen lähdekoodin LLM-malleihin.

Tämä malli on päivitys aiempaan Llama 3 -versioon ja tarjoaa:

- Suuremman kontekstin ikkunan – 128k tokenia vs 8k tokenia
- Suuremman maksimimäärän ulostulotokeneita – 4096 vs 2048
- Parempi monikielinen tuki – johtuen suuremmasta määrästä opetustokeneita

Näiden ansiosta Llama 3.1 pystyy käsittelemään monimutkaisempia käyttötapauksia GenAI-sovelluksissa, kuten:
- Luontainen funktiokutsuminen – mahdollisuus kutsua ulkoisia työkaluja ja funktioita LLM-työnkulun ulkopuolella
- Parempi RAG-suorituskyky – suuremman kontekstin ikkunan ansiosta
- Synteettisen datan generointi – mahdollisuus luoda tehokasta dataa esimerkiksi hienosäätöä varten


### Native Function Calling

Llama 3.1 on hienosäädetty toimimaan tehokkaammin funktio- ja työkalukutsujen kanssa. Mallissa on myös kaksi sisäänrakennettua työkalua, jotka se osaa tunnistaa käyttäjän kehotteen perusteella tarpeellisiksi. Nämä työkalut ovat:

- **Brave Search** – Voidaan käyttää ajantasaisen tiedon, kuten sään, hakemiseen verkosta
- **Wolfram Alpha** – Soveltuu monimutkaisempiin matemaattisiin laskelmiin, joten omien funktioiden kirjoittaminen ei ole välttämätöntä.

Voit myös luoda omia mukautettuja työkaluja, joita LLM voi kutsua.

Alla olevassa koodiesimerkissä:

- Määrittelemme käytettävissä olevat työkalut (brave_search, wolfram_alpha) järjestelmäkehotteessa.
- Lähetämme käyttäjän kehotteen, jossa kysytään tietyn kaupungin säästä.
- LLM vastaa työkalukutsulla Brave Search -työkalulle, joka näyttää tältä: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Huom: Tämä esimerkki tekee ainoastaan työkalukutsun. Jos haluat saada tulokset, sinun täytyy luoda ilmainen tili Brave API -sivulla ja määritellä itse funktio.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Vaikka Llama 3.1 on suuri kielimalli, sen rajoituksiin kuuluu multimodaalisuuden puute. Eli se ei pysty käyttämään erilaisia syötteitä, kuten kuvia, kehotteina ja antamaan niihin vastauksia. Tämä ominaisuus on yksi Llama 3.2:n tärkeimmistä uudistuksista. Näihin ominaisuuksiin kuuluvat myös:

- Multimodaalisuus – pystyy käsittelemään sekä teksti- että kuvakehotteita
- Pienet ja keskikokoiset mallit (11B ja 90B) – tarjoaa joustavia käyttöönotto­vaihtoehtoja
- Vain tekstiin perustuvat mallit (1B ja 3B) – mahdollistaa mallin käytön reunalaitteissa ja mobiililaitteissa sekä tarjoaa matalan viiveen

Multimodaalituki on merkittävä edistysaskel avoimen lähdekoodin mallien maailmassa. Alla olevassa esimerkissä sekä kuva että tekstikehote annetaan Llama 3.2 90B -mallille, joka analysoi kuvan.

### Multimodaalituki Llama 3.2:n kanssa


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Oppiminen ei lopu tähän – jatka matkaa

Kun olet suorittanut tämän oppitunnin, tutustu [Generative AI Learning -kokoelmaamme](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ja jatka Generatiivisen tekoälyn osaamisesi kehittämistä!



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta huomioithan, että automaattiset käännökset saattavat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisissä tapauksissa suosittelemme ammattimaisen ihmiskääntäjän käyttöä. Emme ole vastuussa tämän käännöksen käytöstä mahdollisesti aiheutuvista väärinkäsityksistä tai tulkinnoista.
